# 环境准备

## 监控命令

```
watch -n 1 "ps -Leo pcpu,pid,tid,cmd | sort -nrk 1 | head -n 30"
```

## MiniConda

Guideline: https://docs.anaconda.com/miniconda/

```
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

source ~/miniconda3/bin/activate

conda init --all
```

## VSCode & Jupyter

```
pip install ipykernel -U --user --force-reinstall
```


In [ ]:
print('Hello Python')


hi


## Check Version

```
# 查询镜像中的cuda版本
ldconfig -p | grep cuda

# 查询镜像中的cudnn版本
ldconfig -p | grep cudnn

conda list | grep cudatoolkit

conda list | grep cudnn

```

KB: 用 conda 安装

```
conda search cudatoolkit

conda install cudatoolkit==xx.xx
conda install cudnn==xx.xx

```

KB: 下载安装

https://developer.nvidia.com/cuda-toolkit-archive

https://developer.nvidia.com/cudnn

安装方法：

先解压， 后将动态链接库和头文件放入相应目录

```
mv cuda/include/* /usr/local/cuda/include/
chmod +x cuda/lib64/* && mv cuda/lib64/* /usr/local/cuda/lib64/
```

安装完成以后，增加环境变量：

```
echo "export LD_LIBRARY_PATH=/usr/local/cuda/lib64/:${LD_LIBRARY_PATH} \n" >> ~/.bashrc
source ~/.bashrc && ldconfig
```

提示：

默认镜像都内置了最原生的CUDA和cuDNN，如果您自己安装了cudatoolkits等，那么一般会默认优先使用conda中安装的cudatoolkits，



## Pytorch & Cuda

Ref: https://pytorch.org/get-started/previous-versions/

```
conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.6 -c pytorch -c nvidia
```

#### 替换方案

用 micromambma

```
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
```

root@autodl-container-2e994eb93f-ac29d12f:~# micromamba activate

'micromamba' is running as a subprocess and can't modify the parent shell.
Thus you must initialize your shell before using activate and deactivate.

To initialize the current bash shell, run:
    $ eval "$(micromamba shell hook --shell bash)"
and then activate or deactivate with:
    $ micromamba activate
To automatically initialize all future (bash) shells, run:
    $ micromamba shell init --shell bash --root-prefix=~/.local/share/mamba
If your shell was already initialized, reinitialize your shell with:
    $ micromamba shell reinit --shell bash
Otherwise, this may be an issue. In the meantime you can run commands. See:
    $ micromamba run --help

Supported shells are {bash, zsh, csh, posix, xonsh, cmd.exe, powershell, fish, nu}.
critical libmamba Shell not initialized
root@autodl-container-2e994eb93f-ac29d12f:~#

```
warning  libmamba You have not set the root prefix environment variable.
    To permanently modify the root prefix location, either:
      - set the 'MAMBA_ROOT_PREFIX' environment variable
      - use the '-r,--root-prefix' CLI option
      - use 'micromamba shell init ...' to initialize your shell
        (then restart or source the contents of the shell init script)
    Continuing with default value: "/root/.local/share/mamba"
```

然后替换执行命令：

```
micromamba install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.6 -c pytorch -c nvidia
```



### ffmpeg libsm6 ninja-build libglib

```
apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub \
    && apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/7fa2af80.pub \
    && apt-get update \
    && apt-get install -y ffmpeg libsm6 libxext6 git ninja-build libglib2.0-0 libsm6 libxrender-dev libxext6
```

### openmmlab

```
micromamba install --no-deps \
    mmengine==0.7.3 \
    mmdet==3.0.0 \
    mmsegmentation==1.0.0 \
    git+https://github.com/open-mmlab/mmdetection3d.git@22aaa47fdb53ce1870ff92cb7e3f96ae38d17f61
```

### mmcv 2.0

```
pip install mmcv==2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html --no-deps
```

### MinkowskiEngine

MinkowskiEngine 是一个用于稀疏张量的自动微分库。

Install MinkowskiEngine

Feel free to skip nvidia-cuda-dev if minkowski installation is fine

```
＃ 只有当后面的命令执行失败后，下面的命令才需要执行。
# apt-get update \
# && apt-get -y install libopenblas-dev nvidia-cuda-dev

TORCH_CUDA_ARCH_LIST="6.1 7.0 8.6" \
pip install git+https://github.com/NVIDIA/MinkowskiEngine.git@02fc608bea4c0549b0a7b00ca1bf15dee4a0b228 -v --no-deps \
  --install-option="--blas=openblas" \
  --install-option="--force_cuda"
```

现在 pip 已经不支持 --install-option 参数了。

替换为：

```
BLAS=openblas \
FORCE_CUDA=true \
TORCH_CUDA_ARCH_LIST="6.1 7.0 8.6"

pip install git+https://github.com/NVIDIA/MinkowskiEngine.git@02fc608bea4c0549b0a7b00ca1bf15dee4a0b228 -v --no-deps

```

安装还是失败，换方法。

参考： https://blog.csdn.net/m0_60197472/article/details/125293739


```
pip install ninja

git clone https://github.com/NVIDIA/MinkowskiEngine.git

echo $CONDA_PREFIX

python setup.py install --blas_include_dirs=${CONDA_PREFIX}/include --blas=openblas
```

In [ ]:
# Check

import MinkowskiEngine as ME
print(ME.__version__)


### torch-scatter

```
pip install torch-scatter==2.1.2 -f https://data.pyg.org/whl/torch-1.13.0+cu116.html --no-deps
```

## ScanNet superpoint segmentator

```
git clone https://github.com/Karbo123/segmentator.git

cd segmentator/csrc

git reset --hard 76efe46d03dd27afa78df972b17d07f2c6cfb696

mkdir build

cd build

cmake .. \
    -DCMAKE_PREFIX_PATH=`python -c 'import torch;print(torch.utils.cmake_prefix_path)'` \
    -DPYTHON_INCLUDE_DIR=$(python -c "from distutils.sysconfig import get_python_inc; print(get_python_inc())") \
    -DPYTHON_LIBRARY=$(python -c "import distutils.sysconfig as sysconfig; print(sysconfig.get_config_var('LIBDIR'))") \
    -DCMAKE_INSTALL_PREFIX=`python -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib())'`



cmake 报错

```
CMake Error at CMakeLists.txt:2 (cmake_minimum_required):
  CMake 3.18 or higher is required.  You are running version 3.16.3
```



解决方法1

Ref: https://askubuntu.com/questions/355565/how-do-i-install-the-latest-version-of-cmake-from-the-command-line

```
apt remove --purge --auto-remove cmake

apt update

apt install -y software-properties-common lsb-release

apt clean all

wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | gpg --dearmor - | sudo tee /etc/apt/trusted.gpg.d/kitware.gpg >/dev/null

apt-add-repository "deb https://apt.kitware.com/ubuntu/ $(lsb_release -cs) main"

apt update

apt install kitware-archive-keyring

rm /etc/apt/trusted.gpg.d/kitware.gpg

apt update

apt install cmake

cmake --version

cmake version 3.30.2

```

然后回到前面，继续执行 cmake 。



make

make install

cd ../../..
```

## 最后的一堆组件包

```
pip install --no-deps \
    spconv-cu116==2.3.6 \
    addict==2.4.0 \
    yapf==0.33.0 \
    termcolor==2.3.0 \
    packaging==23.1 \
    numpy==1.24.1 \
    rich==13.3.5 \
    opencv-python==4.7.0.72 \
    pycocotools==2.0.6 \
    Shapely==1.8.5 \
    scipy==1.10.1 \
    terminaltables==3.1.10 \
    numba==0.57.0 \
    llvmlite==0.40.0 \
    pccm==0.4.7 \
    ccimport==0.4.2 \
    pybind11==2.10.4 \
    ninja==1.11.1 \
    lark==1.1.5 \
    cumm-cu116==0.4.9 \
    pyquaternion==0.9.9 \
    lyft-dataset-sdk==0.0.8 \
    pandas==2.0.1 \
    python-dateutil==2.8.2 \
    matplotlib==3.5.2 \
    pyparsing==3.0.9 \
    cycler==0.11.0 \
    kiwisolver==1.4.4 \
    scikit-learn==1.2.2 \
    joblib==1.2.0 \
    threadpoolctl==3.1.0 \
    cachetools==5.3.0 \
    nuscenes-devkit==1.1.10 \
    trimesh==3.21.6 \
    open3d==0.17.0 \
    plotly==5.18.0 \
    dash==2.14.2 \
    plyfile==1.0.2 \
    flask==3.0.0 \
    werkzeug==3.0.1 \
    click==8.1.7 \
    blinker==1.7.0 \
    itsdangerous==2.1.2 \
    importlib_metadata==2.1.2 \
    zipp==3.17.0
```

## 数据集

## Scannet

使用下载好的 download_scannet.py 文件来执行下载任务。


```
#下载scannet_frames_25k
python download_scannetv2.py -o data --preprocessed_frames

#下载scannet_frames_test
python download_scannetv2.py -o data --test_frames_2d

```

将下载后的 zip 文件解压、并移动到 data/scannet 下。

``` shell
unzip scannet_frames_25k.zip -d data/scannet
```

最后发现，这个文件里只有图片，标记。 对我来说，没用。

```
python download_scannet2.py -o /home/content/git/oneformer3d/data/scannet --type _vh_clean_2.ply
python download_scannet2.py -o /home/content/git/oneformer3d/data/scannet --type _vh_clean_2.labels.ply
python download_scannet2.py -o /home/content/git/oneformer3d/data/scannet --type _vh_clean_2.0.010000.segs.json
python download_scannet2.py -o /home/content/git/oneformer3d/data/scannet --type .aggregation.json
```

### 数据预处理

#### 安装缺的python组件

```shell
micromamba install markupsafe
micromamba install jinja2
```



#### 预加载数据处理

```shell

cd data/scannet

python batch_load_scannet_data.py

python tools/create_data.py scannet --root-path ./data/scannet --out-dir ./data/scannet --extra-tag scannet

```


```
mkdir /home/content/git/oneformer3d/work_dirs/tmp

cp sstnet_scannet.pth /home/content/git/oneformer3d/work_dirs/tmp

export PYTHONPATH=/home/content/git/oneformer3d




```


在我的环境中，命令如下

```
cd /home/content/git/oneformer3d

python tools/train.py configs/oneformer3d_1xb4_scannet.py

# train (with validation)
python tools/train.py configs/oneformer3d_1xb4_scannet.py

# test

# 先执行 
python tools/fix_spconv_checkpoint.py \
    --in-path work_dirs/oneformer3d_1xb4_scannet/epoch_19.pth \
    --out-path work_dirs/oneformer3d_1xb4_scannet/epoch_19.pth

export PYTHONPATH=/home/content/git/oneformer3d

python tools/test.py configs/oneformer3d_1xb4_scannet.py work_dirs/oneformer3d_1xb4_scannet/epoch_19.pth

```

# 训练

## 下载预训练模型

### ScanNet 扫描

https://github.com/oneformer3d/oneformer3d/releases/download/v1.0/oneformer3d_1xb4_scannet.pth

https://github.com/Eric-Zhong/oneformer3d/blob/main/configs/oneformer3d_1xb4_scannet.py


### S3DIS

https://github.com/oneformer3d/oneformer3d/releases/download/v1.0/oneformer3d_1xb2_s3dis-area-5.pth

https://github.com/Eric-Zhong/oneformer3d/blob/main/configs/oneformer3d_1xb2_s3dis-area-5.py


## 训练

```
# pre-train
python tools/train.py configs/instance-only-oneformer3d_1xb2_scannet-and-structured3d.py

python tools/fix_spconv_checkpoint.py \
    --in-path work_dirs/instance-only-oneformer3d_1xb2_scannet-and-structured3d/iter_600000.pth \
    --out-path work_dirs/tmp/instance-only-oneformer3d_1xb2_scannet-and-structured3d.pth
```


## 训练评估

```
export PYTHONHOME=/home/content/git/oneformer3d

train (with validation)
python tools/train.py configs/oneformer3d_1xb2_s3dis-area-5.py
```

## 测试

```
# test
python tools/fix_spconv_checkpoint.py \
    --in-path work_dirs/oneformer3d_1xb2_s3dis-area-5/epoch_512.pth \
    --out-path work_dirs/oneformer3d_1xb2_s3dis-area-5/epoch_512.pth
python tools/test.py configs/oneformer3d_1xb2_s3dis-area-5.py \
    work_dirs/oneformer3d_1xb2_s3dis-area-5/epoch_512.pth

```